In [18]:
%load_ext autoreload
%autoreload 2

import torch
import nltk
from torch.utils.data import Dataset, DataLoader
import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
train_df = pd.read_csv('../data/raw/train.csv')
test_df = pd.read_csv('../data/raw/test.csv')

In [20]:
train_df.head()

,textID,text,selected_text,sentiment
0,28ac06f416,good luck with your auction,good luck with your auction,positive
1,92098cf9a7,Hmm..You can`t judge a book by looking at its ...,Hmm..You can`t judge a book by looking at its ...,neutral
2,7858ff28f2,"Hello, yourself. Enjoy London. Watch out for ...",They`re mental.,negative
3,b0c9c67f32,We can`t even call you from belgium sucks,m suck,negative
4,7b36e9e7a5,not so good mood..,not so good mood..,negative


In [21]:
from src.features.preprocess import preprocess
from tqdm.notebook import tqdm
tqdm.pandas()

train_df["text"] = train_df["text"].progress_apply(lambda x: preprocess(x))
test_df["text"] = train_df["text"].progress_apply(lambda x: preprocess(x))

  0%|          | 0/24732 [00:00<?, ?it/s]

  0%|          | 0/24732 [00:00<?, ?it/s]

In [22]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(train_df, test_size=0.1, random_state=0)

In [23]:
target_conversion = {
    'neutral': 0,
    'positive': 1,
    'negative': -1
}

train_df['target'] = train_df['sentiment'].map(target_conversion)
val_df['target'] = val_df['sentiment'].map(target_conversion)

In [24]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()

X_train_counts = count_vect.fit_transform(list(train_df['text'].values))
X_val_counts = count_vect.transform(list(val_df['text'].values))
X_test_counts = count_vect.transform(list(test_df['text'].values))

In [25]:
print('Train feature shape:', X_train_counts.shape)
print('Train feature shape:', X_val_counts.shape)
print('Test feature shape:', X_test_counts.shape)

Train feature shape: (22258, 22201)
Train feature shape: (2474, 22201)
Test feature shape: (2748, 22201)


In [26]:
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB().fit(X_train_counts, train_df['target'])
val_predictions_nb = clf.predict(X_val_counts)

In [27]:
accuracy = (val_predictions_nb == val_df['target'].values).mean()
print('The accuracy of our multinomial Naive Bayes classifier is: {:.2f}%'.format(accuracy*100))

The accuracy of our multinomial Naive Bayes classifier is: 66.13%


In [17]:
from sklearn.metrics import fbeta_score

fbeta = fbeta_score(val_df['target'].values, val_predictions_nb, average='macro', beta=1.0)
print('The fbeta score is:', fbeta)

The fbeta score is: 0.6413732952519994


In [36]:
# Creating a submission

X_train_counts = count_vect.fit_transform(list(train_df['text'].values) + list(val_df['text'].values))
X_test_counts = count_vect.transform(list(test_df['text'].values))

clf = MultinomialNB().fit(X_train_counts, np.hstack([train_df['target'].values, val_df['target'].values]))
test_predictions_nb = clf.predict(X_test_counts)

submission_df = pd.DataFrame()
submission_df['textID'] = test_df['textID']
submission_df['sentiment'] = test_predictions_nb
submission_df.to_csv('TA_baseline_NB.csv', index=False)